In [51]:
import pandas as pd

In [52]:
df = pd.read_csv("dpschool2a.csv", delimiter=";")

In [53]:
df = df[['title', 'short_description', 'description', 'latitude', 'longitude', 'address', 'addr_no', 
         'zip_code', 'city', 'country', 'additional_fields', 'image', 'icons', 'modified']]

In [54]:
import json
df["additional_fields"] = df["additional_fields"].map(lambda x: json.loads(x))
df["image"] = df["image"].map(lambda x: json.loads(x))

In [55]:
df = df[df['latitude'].notna()]

In [56]:
from pandas import isnull
df["icons"] = df["icons"].map(lambda x: {} if isnull(x) else x)
df["icons"] = df["icons"].map(lambda x: x if isinstance(x, str) else str(x))
df["icons"] = df["icons"].map(lambda x: json.loads(x))

In [57]:
df["modified"] = pd.to_datetime(df["modified"], errors="coerce")

In [58]:
df['longitude'] = df['longitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000
df['latitude'] = df['latitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000

In [59]:
df["description"] = df["description"].map(lambda x: "" if isnull(x) else x)
df["short_description"] = df["short_description"].map(lambda x: "" if isnull(x) else x)
df["addr_no"] = df["addr_no"].map(lambda x: "" if isnull(x) else x)

In [60]:
df["address"] = df["address"] + " " + df["addr_no"]

In [61]:
df.drop(["addr_no"], axis=1, inplace=True)

In [62]:
df["position"] = None
for i in df.index:
    df.at[i, 'position'] = (df.loc[i]['latitude'], df.loc[i]['longitude'])

In [63]:
icons = {"Toilette für Alle": 10, "Rolli WC": 11, "Rolli WC -DIN fern": 20, "WC eng": 17, "kein Rolli WC vorhanden": 21,
        "Parken": 7, "Stufe": 1, "WC - unisex": 4, "WC getrennt": 6, "Rollstuhl": 5, "Lift groß": 12, "Lift klein": 13,
        "Kino-Untertitel": 15, "Induktionsschleife": 14, "low-vision": 18, "kognitiv beeinträchtigt": 19, "Umkleide": 9,
        "Dusche": 8, "nicht rollstuhlgerecht": 16, "Rampe": 2, "keine Steigung - eben": 26, "moderate Steigung / Gefälle": 22,
        "weitere moderate Steigungen / Gefälle": 23, "heftige Steigung / Gefälle": 25, "weitere heftige Steigung / Gefälle": 24}
new_dict = {v: k for k, v in icons.items()}
new_dict.update({40: "Euro Key", 41: "Ramp Incline", 42: "Door Width"})

In [64]:
df["icons"] = df["icons"].map(lambda x: tuple(map(int, list(x.values()))))

In [65]:
df[40] = df["additional_fields"].map(lambda x: x["wc_euro"] if "wc_euro" in x.keys() else "0")
df[40] = df[40].map(lambda x: int(x) if x in ["1", "0"] else 0)

In [66]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

df[41] = df["additional_fields"].map(lambda x: x["wc_ramp_incline"])
df[41] = df[41].map(lambda x: x.replace(",","."))
df[41] = df[41].map(lambda x: float(x) if isfloat(x) else 0)
#need to check further as some have extra text

In [67]:
df[42] = df["additional_fields"].map(lambda x: x["wc_door_width"])
df[42] = df[42].map(lambda x: int(x) if x != "" else 150)
df[42] = df[42].map(lambda x: x if x > 15 else x*10)

In [68]:
x = pd.get_dummies(df.icons.apply(pd.Series).stack()).sum(level=0)
x = x.rename(int, axis="columns")
col = list(x.columns)
for i in [40, 41, 42]:
    col.append(i)

In [69]:
df = df.rename(columns={40:"eurokey", 41: "ramp_steepness", 42: "door_width"})

In [70]:
# df.drop(["icons"], axis=1, inplace=True)
# df = pd.concat([df, x], axis=1)

In [71]:
# def check(series):
#     x = {}
#     for i in series.index:
#         if i in col:
#             x.update({i: series[i]})
#         else:
#             continue
#     return x

In [72]:
# df["options"] = df.apply(lambda x: check(x), axis=1)

In [73]:
# df.drop(col, axis=1, inplace=True)

In [74]:
# columns = pd.DataFrame.from_dict(new_dict, orient="index", columns=["option_name"])
# columns.index.name = "id"

In [75]:
# columns.to_csv("options.csv")

In [76]:
df["photo"] = df["image"].map(lambda x: x["image_small"])
df["plan"] = df["additional_fields"].map(lambda x: "images/wc-plan/" + x["wc_plan"] + ".jpg")
#df["seat_height"] = df["additional_fields"].map(lambda x: x["wc_seat"])

In [77]:
#df["seat_height"] = df["seat_height"].map(lambda x: x[:2] if x != "" else None)

In [78]:
df = df.drop(df.loc[df['icons'].isin([(21,)])].index)

In [79]:
df["grip"] = df["additional_fields"].map(lambda x:x["wc_wheregrip"])

In [80]:
df["grip"].unique()

array(['lr', 'l', 'rfl', 'fr', 'none', 'lfr', 'flr', 'r', 'fl'],
      dtype=object)

lr = flexible left & flexible right

l = flexible left

rfl & flr = fixed left and flexible right

fr = fixed right

none = no grip

lfr = flexible left and fixed right

r = flexible right

fl = fixed left

In [81]:
df = df[["title", "short_description", "photo", "plan", "position", "address", "eurokey", "ramp_steepness", "door_width", "zip_code", "city", "grip"]]

In [82]:
errors = pd.read_excel("toilets_text errors.xlsx")

In [83]:
errors.dropna(inplace=True)

In [84]:
list(errors["Text error"])

['Ãœ', 'Ã¼', 'Ã–', 'Ã¶', 'Ã„', 'Ã¤', 'ÃŸ', 'Ã©', 'Ã¨', 'Ã‡', 'â€“']

In [85]:
import re

errors = pd.Series(errors["correction"].values,index=errors["Text error"]).to_dict()

# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in errors.items()) 
#Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
pattern = re.compile("|".join(rep.keys()))

In [86]:
df["short_description"] = df["short_description"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["title"] = df["title"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["address"] = df["address"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["city"] = df["city"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))

In [87]:
df = df.drop(df.loc[df['icons'].isin([(21,)])].index)

In [88]:
df.drop([315, 1091, 20, 1090, 1088, 1086, 1087, 1089], inplace=True)
df.to_json("toilets_v9.json")

In [105]:
df[df['grip'].str.contains(("|").join(["lr"]))]

,title,short_description,photo,plan,position,address,eurokey,ramp_steepness,door_width,zip_code,city,grip,id
1,WC Bad Georgenschwaige Damen,Bedingt öffentliche Toilette in einem Freibad.,images/wc-af/ba-george1WC1f1.jpg,images/wc-plan/ba-george1WC1.jpg,"(48.1756, 11.5717)",Belgradstr. 195,0,0.0,85,80804,München,lr,1
3,WC Freibad Unterhaching,Bedingt öffentliche Toilette mit Dusche in ein...,images/wc-af/ba-unterh1WC1f1.jpg,images/wc-plan/ba-unterh1WC.jpg,"(48.0619, 11.6208)",Schrenkstr. 2,1,0.0,75,82008,Unterhaching,lr,3
4,WC U-Bahnhof Münchner Freiheit,"Öffentliche Toilette im U-Bahnhof, zugänglich ...",images/wc-af/bf-muefre1WC1f1.jpg,images/wc-plan/bf-muefre1WC1.jpg,"(48.1622, 11.587)",Münchner Freiheit,1,0.0,85,80802,München,lr,4
6,WC Riemer See,Öffentliche Toilette mit Pflegeliege am Badese...,images/wc-af/ba-riemer1WC1f1.jpg,images/wc-plan/ba-riemer1WC1.jpg,"(48.1271, 11.7078)",Riemer See,1,0.0,95,81829,München,lr,6
7,WC Ostbahnhof München,"Öffentliche Toilette im Bahnhof, zugänglich mi...",images/wc-af/bf-ostbah1fWC2.jpg,images/wc-plan/bf-ostbah1WC1.jpg,"(48.1284, 11.6026)",Orleansplatz,1,0.0,88,81667,München,lr,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,"WC Boulderwelt München Süd, Brunnthal",Halböffentliche Toilette in einer Sportstätte.,images/wc-nz/sp-boulde1WC1f2.jpg,images/wc-plan/sp-boulde1WC1.jpg,"(48.0403, 11.6639)",Eugen-Sänger-Ring 4,0,0.0,90,85649,Brunnthal,lr,1085
1086,WC Volkstheater mit Pflegeliege,Halböffentliches WC an einem Veranstaltungsort.,images/wc-nz/ve-volkst1WC1f3.jpg,images/wc-plan/ve-volkst1WC1.jpg,"(0.0, 0.0)",Tumblingerstr. 29,0,0.0,110,80337,München,lr,1086
1087,WC Schmock im Volkstheater,Halböffentliche Toilette in einer Gaststätte.,images/wc-nz/schmoc1WC1f2.jpg,images/wc-plan/schmoc1WC.jpg,"(0.0, 0.0)",Tumblingerstr. 29,0,0.0,100,80337,München,lr,1087
1088,"WC Musikhochschule, Reaktorhalle und Carl-Orff...",Halböffentliche Toilette im Hochschulgebäude m...,images/wc-af/div-musikh1WC1f3.jpg,images/wc-plan/div-musikh1WC1.jpg,"(0.0, 0.0)",Luisenstr. 37a,0,0.0,90,80333,München,lr,1088


In [67]:
#def to_1D(series):
 #return pd.Series([x for _list in series for x in _list])